# Recipes and Ratings

**Name(s)**: Kang Lee, Jacob Kavanal

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import *

## Introduction

In [2]:
# Read in files
recipes = pd.read_csv('data/RAW_recipes.csv')
ratings = pd.read_csv('data/interactions.csv')

In [3]:
recipes

,name,id,minutes,contributor_id,...,steps,description,ingredients,n_ingredients
0,1 brownies in the world best ever,333281,40,985201,...,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9
1,1 in canada chocolate chip cookies,453467,45,1848091,...,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11
2,412 broccoli casserole,306168,40,50969,...,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9
...,...,...,...,...,...,...,...,...,...
83779,zydeco ya ya deviled eggs,308080,40,37779,...,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
83780,cookies by design cookies on a stick,298512,29,506822,...,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10
83781,cookies by design sugar shortbread cookies,298509,20,506822,...,"['whip sugar and shortening in a large bowl , ...","i've heard of the 'cookies by design' company,...","['granulated sugar', 'shortening', 'eggs', 'fl...",7


In [4]:
ratings

,user_id,recipe_id,date,rating,review
0,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
1,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
2,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
731924,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
731925,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...
731926,2001868099,78003,2017-12-18,5,I am so glad I googled and found this here. Th...


In [5]:
# Merge dataframes
merged = recipes.merge(ratings, how="left", left_on="id", right_on="recipe_id")
merged['rating'] = merged['rating'].replace(0, np.nan)

In [6]:
# Find average rating per recipe
avg_ratings = merged.groupby('id')['rating'].mean()
avg_ratings_df = avg_ratings.reset_index().rename(columns={'rating': 'avg_rating'})

recipes = recipes.merge(avg_ratings_df, how='left', on='id')

## Data Cleaning and Exploratory Data Analysis

Handling Missingness

In [7]:
# Finding columns with missing vals
nan_columns = recipes.columns[recipes.isnull().any()]

print(nan_columns)

Index(['name', 'description', 'avg_rating'], dtype='object')


In [8]:
recipes[recipes['name'].isna()] # 1 Missing
recipes[recipes['description'].isna()] # 70 Missing

,name,id,minutes,contributor_id,...,description,ingredients,n_ingredients,avg_rating
1486,almond cookie bites,401761,16,1434673,...,NaN,"['all-purpose flour', ""fisher chef's naturals ...",9,2.67
3087,apricot gorgonzola crescent appetizers,332410,40,991676,...,NaN,['pillsbury refrigerated crescent dinner rolls...,6,4.67
3685,asparagus milanese,382664,15,714468,...,NaN,"['asparagus', 'parmigiano-reggiano cheese', 'b...",5,4.50
...,...,...,...,...,...,...,...,...,...
81188,wasatch mountain chili,290480,50,778477,...,NaN,"['onion', 'olive oil', 'hominy', 'great northe...",14,5.00
81701,white bean chicken chili giada de laurentiis,430591,75,714468,...,NaN,"['olive oil', 'onion', 'garlic cloves', 'groun...",18,5.00
83070,yukon gold potatoes jacques pepin style,387006,20,714468,...,NaN,"['yukon gold potatoes', 'salt', 'fresh ground ...",6,4.00


In [9]:
# Find missing name
recipes.iloc[238]['steps']


"['in a bowl , combine ingredients except for olive oil', 'slowly whisk inches', 'olive oil until thickened', 'great with field greens', 'makes about 2 / 3', 'cup dressing']"

In [10]:
# Fill in missing name
recipes['name'].iloc[238] = 'Salad Vinegarette'

/var/folders/1k/2p71sg_x20q3mg91vw9xjqbw0000gn/T/ipykernel_7585/4253574573.py:2: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




In [11]:
# Fill missing descriptions with a placeholder
recipes['description'] = recipes['description'].fillna('No description available')

print(f"Number of missing descriptions after filling: {recipes['description'].isna().sum()}")


Number of missing descriptions after filling: 0


In [12]:
nan_columns = recipes.columns[recipes.isnull().any()]

print(nan_columns)

Index(['avg_rating'], dtype='object')

In [13]:
# Check missing values in Average Rating
recipes[recipes['avg_rating'].isna()]

,name,id,minutes,contributor_id,...,description,ingredients,n_ingredients,avg_rating
10,lplermagronen,455351,55,1308592,...,"known as swiss mac n cheese, älplermagronen wa...","['potato', 'penne pasta', 'onions', 'butter', ...",8,NaN
14,der wiener schnitzel style chili dog sauce,523359,55,2000414501,...,this was the best chili dog ever invented! i l...,"['ground beef', 'ground pork', 'water', 'corns...",14,NaN
64,boo tiful jell o cups,505132,50,2549237,...,this was so good...everyone loved them.,"['boiling water', 'orange gelatin', 'ice cubes...",5,NaN
...,...,...,...,...,...,...,...,...,...
83631,zucchini lemon poppyseed bread,517690,80,508997,...,i wanted to try something a little different w...,"['sugar', 'eggs', 'vanilla', 'vegetable oil', ...",14,NaN
83651,zucchini pancakes with a difference,314963,25,27706,...,this started as a recipe from a mexican cookbo...,"['zucchini', 'corn', 'jalapeno', 'egg', 'bisqu...",8,NaN
83737,zucchini oat bread,292835,105,324390,...,the bh&g $400 winner of the bread and rolls ca...,"['nonstick cooking spray', 'sugar', 'ground ci...",15,NaN


Decided to leave as NaN, some recipes don't have any ratings

Exploding Nutritional Information

In [14]:
# Convert nutrition column from string to list
recipes['nutrition'] = recipes['nutrition'].apply(eval)


In [15]:
# Convert nutrition list into separate columns
nutrition_df = pd.DataFrame(recipes['nutrition'].tolist(), 
                          columns=['calories', 'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)', 'saturated fat (PDV)', 'carbohydrates (PDV)'])

# Add the nutrition columns to the original dataframe
recipes = pd.concat([recipes, nutrition_df], axis=1)


Converting Times to DateTime

In [16]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])

In [17]:
recipes.head()

,name,id,minutes,contributor_id,...,sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,1 brownies in the world best ever,333281,40,985201,...,3.0,3.0,19.0,6.0
1,1 in canada chocolate chip cookies,453467,45,1848091,...,22.0,13.0,51.0,26.0
2,412 broccoli casserole,306168,40,50969,...,32.0,22.0,36.0,3.0
3,millionaire pound cake,286009,120,461724,...,13.0,20.0,123.0,39.0
4,2000 meatloaf,475785,90,2202916,...,12.0,29.0,48.0,2.0


## Assessment of Missingness

In [18]:
# to do

## Hypothesis Testing

In [19]:
# to do

## Framing a Prediction Problem

In [20]:
# to do

## Baseline Model

In [21]:
# to do

## Final Model

In [22]:
# to do

## Fairness Analysis

In [23]:
# to do